In [4]:
import os 
import pandas as pd 
import numpy as np 
import seaborn as sns 
import mne 
import torch
import torch.nn as nn
import math
from scipy.interpolate import interp1d
from scipy.stats import iqr 

from typing import Optional, Dict, Any, Tuple

import lightning as L 
from lightning.pytorch.callbacks import ModelCheckpoint

from sklearn.model_selection import train_test_split
from lightning.pytorch.callbacks.early_stopping import EarlyStopping


from torch.utils.data import ConcatDataset, Dataset, DataLoader, RandomSampler


In [5]:
def is_integer_with_custom_precision(num: float, eps: float = 1e-6, int_value: Optional[int] = None):
    if int_value is None:
        int_value = int(num)
    return abs(int_value - num) < eps, int_value



In [6]:
EEG_SAMPLING_RATE = 250
FMRI_TR = 2.0  # sec

RUNS = [
    ('rs', '01'),
    ('rs', '02'),
    ('rs', '03'),
    ('rs', '04'),
    ('rs', '05'),
    ('rs', '06'),
    ('socword', '01'), 
    ('socword', '02'),
    ('socword', '03'),
    ('socword', '04'),
    ('socword', '05'),
    ('socword', '06')
    ]
RUNSNATVIEW = [
    ('01', 'checker'),
    ('01', 'dme_run-01'),
    ('01', 'dme_run-02'),
    ('01', 'inscapes'),
    ('01', 'monkey1_run-01'),
    ('01', 'monkey1_run-02'),
    ('01', 'rest'),
    ('01', 'tp_run-01'),
    ('01', 'tp_run-02')
]

RUNS_1895 = [
    ('rs', '01'),
    ('rs', '02'),
    ('socword', '01'), 
    ('socword', '02'),
    ('socword', '03')
    ]
# IN SAMPLES WITH 2 Hz SAMPLING RATE. FOR SECONDS DIVIDE BY 2
TEST_SIZES = {
    'checker': 100,
    'dme': 250,
    'inscapes': 250,
    'monkey1': 150,
    'rest': 250,
    'tp': 100,
    'dmh': 250,
    'monkey2': 150,
    'monkey5': 150
}


In [7]:
separate_global_trend = True  

In [8]:
def load_eeg_data(root: str, ses:str, run:str, task: str, eeg_channels: Optional[list[str]]):

    filename = f'sub-101472_ses-{ses}_task-{task}_run-{run}_eegMRbvCBbviiR250.vhdr'
    eeg_vhdr_path = os.path.join(root, f'ses-{ses}', 'eeg', filename)

    
    eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
    events, event_ids = mne.events_from_annotations(eeg_raw)
    events_dict_cut = {'Scanner/Scan Start': event_ids['Scanner/Scan Start']}  # use only Scan Start event


    try:
        scan_key = 'Scanner/Scan Start'
        segment_scans = events[events[:, -1] == event_ids[scan_key]]
    except: 
        scan_key = 'Stimulus/Scan Start'
        segment_scans = events[events[:, -1] == event_ids[scan_key]] 
        print(f'ERROR: FAULTY EVENT NAME. PATH:{eeg_vhdr_path}')


    unique_scan_diffs, counts_diffs = np.unique(np.diff(segment_scans[:, 0]), return_counts=True)
    assert len(unique_scan_diffs) == 1, (unique_scan_diffs, counts_diffs)
    assert unique_scan_diffs[0] == 3000  

    assert eeg_raw.info['sfreq'] == EEG_SAMPLING_RATE
    first_segment = segment_scans[0, 0] / EEG_SAMPLING_RATE

    cropped_eeg = eeg_raw.crop(tmin=first_segment)
    df_eeg = cropped_eeg.to_data_frame()

    if eeg_channels is None:
        eeg_channels = [column for column in df_eeg.columns if column != 'time']

    eeg_times = df_eeg['time']


    epochs = mne.Epochs(
        cropped_eeg,
        events = events, 
        event_id =events_dict_cut, 
        tmin = 6.5,
        tmax = 11.5-1/EEG_SAMPLING_RATE, 
        baseline = None, 
        preload = True
    )

    epochs_df = epochs.to_data_frame()
    #try: 
    assert (epochs_df['epoch'].unique() == np.where(events[:,2] == 10002)).all()
    #except: 
        #assert (epochs_df['epoch'].unique() == np.where(events[:,2] == 10001)).all()
        #print(f'FAULTY EVENT NAMES. SESSION:{ses},RUN:{run},  TASK:{task}')
    epochs_array = epochs.get_data()[:-1, ...]
    segment_scans = segment_scans[:-1]


    return epochs_array, eeg_channels, eeg_vhdr_path, eeg_times, events,  event_ids, segment_scans

In [9]:
#UPDATE FUNCTIONS BELOW FOR THE EPOCHS RETURN + ALIGN EPOCHT WITH FMRI  

In [10]:
def load_fmri_data(root, ses, task, run, roi_folder, rois, rois_for_global_trend = None): 
    """
    Loads pre-processed fMRI data. The original TR is 2.0 
    """
    filename = f'sub-101472_ses-{ses}_task-{task}_run-{run}.csv'
    csv_path = os.path.join(root, f'ses-{ses}', roi_folder, filename)
    fmri_data = pd.read_csv(csv_path)

    if rois is None:
        rois = [column for column in fmri_data.columns if column not in ['Nvol', ' Tstart']]
    # assert for tsart integer 
    assert all([i.is_integer() for i in fmri_data[' Tstart']])
    fmri_times = fmri_data[' Tstart'].to_numpy()

    fmri_times = fmri_times.astype(int)
    fmri_times = fmri_times//12

    fmri_data = fmri_data.assign(N_cycle=pd.Series(fmri_times).values)
    df_ncycle = fmri_data.groupby(by = 'N_cycle')

    fmri_array = np.full((len(df_ncycle), len(rois), 3), fill_value = np.inf)

    for idx, group in df_ncycle: 
        group = group[rois].to_numpy()
        fmri_array[idx, ...] = group.T 

    # check if no inf that were previously filled are left  
    assert not np.isinf(fmri_array).any(), (f'') 

    # cut the first triplet 
    fmri_array = fmri_array[1:, ...]
    if rois_for_global_trend is None:
        rois_for_global_trend = list(rois)

    all_rois = [roi for roi in list(rois)]
    all_rois = all_rois + [roi for roi in rois_for_global_trend if roi not in rois]


    return fmri_array, fmri_times, rois, csv_path, fmri_data, rois_for_global_trend



In [ ]:
def load_data(root: str, run: str, task: str, ses:str,
              rois: Optional[list[str]], eeg_channels: Optional[list[str]],separate_global_trend: bool,
              sampling_rates_ratio = 500, roi_folder: str = 'roi'):
    
    fmri_array, fmri_times, rois, fmri_path, fmri_data, rois_for_global_trend = load_fmri_data(
        root=root, run=run, ses=ses, task=task, rois=rois, roi_folder=roi_folder
    )

    epochs_array, eeg_channels, eeg_path, eeg_times, events, event_ids, segment_scans = load_eeg_data(
        root=root, ses=ses, run=run, task=task, eeg_channels=eeg_channels
    )

    assert (fmri_array.shape[0]) == len(segment_scans), (f'EEG segment marks should be 3x '
                                                 f'fMRI sample size. '
                                                 f'fMRI sample size is {(fmri_array.shape[0])}. '
                                                 f'Number of segment marks: {len(segment_scans)} '
                                                 f'file is Root: {root},  ses: {ses}, run: {run},'
                                                 f' task: {task}.')
    


   

    # assert epochs.shape[0] == (df_fmri.shape[0] * sampling_rates_ratio), (f'EEG size: {df_eeg.shape}, fMRI '
    #                                                                       f'size: {df_fmri.shape}, sr ratio: '
    #                                                                       f'{sampling_rates_ratio}.\nRoot: {root},  '
    #                                                                       f'sub: {sub}, ses: {ses}, task: {task}.')
    #fmri_for_global_trend = fmri_array.T
    fmri_reshaped = np.transpose(fmri_array, (1, 0, 2))
    n_rois, n_cycles, n_scans = fmri_reshaped.shape
    fmri_reshaped = np.reshape(fmri_reshaped, (n_rois, n_cycles * n_scans))

    if separate_global_trend:
        assert fmri_reshaped.ndim == 2, fmri_reshaped.shape
        global_trend_array = np.mean(fmri_reshaped, axis=0)

        inner = np.inner(global_trend_array, global_trend_array)
        outer = np.outer(global_trend_array, global_trend_array)

        print(inner.shape, outer)

        detrend = fmri_reshaped @ outer / inner
        fmri_reshaped = fmri_reshaped - detrend

 
    fmri_reshaped = np.concatenate((fmri_reshaped, global_trend_array[None, :]), axis=0)
    assert np.equal(fmri_reshaped[-1], global_trend_array).all()

    fmri_reshaped = np.reshape(fmri_reshaped, (n_rois + 1, n_cycles, n_scans))
    fmri_output = np.transpose(fmri_reshaped, (1, 0, 2))

    if not separate_global_trend:
        assert fmri_output.shape == fmri_array.shape, (f'fmri output: {fmri_output.shape}, fmri array: {fmri_array.shape}')
    #fmri_for_global_trend = df_fmri[rois_for_global_trend].to_numpy().T



    # center BOLD around 0
    #fmri_array = fmri_array - np.mean(fmri_array, axis=1, keepdims=True)
    #fmri_for_global_trend = fmri_for_global_trend - np.mean(fmri_for_global_trend, axis=1, keepdims=True)

    # if separate_global_trend:
    #     assert fmri_array.ndim == 2, fmri_array.shape
    #     assert fmri_for_global_trend.ndim == 2, fmri_for_global_trend.shape
    #     global_trend_array = np.mean(fmri_for_global_trend, axis=0)

    #     inner = np.inner(global_trend_array, global_trend_array)
    #     outer = np.outer(global_trend_array, global_trend_array)

    #     detrend = fmri_array @ outer / inner
    #     fmri_array = fmri_array - detrend

    #     for idx in range(fmri_array.shape[0]):
    #         corr_coeff = np.corrcoef(fmri_array[idx, :], global_trend_array)[0, 1]
    #         assert abs(corr_coeff) < 1e-6, (
    #             f'Sub: {sub}, ses: {ses}, task: {task}, ROI: {rois[idx]} has correlation above threshold: {corr_coeff}'
    #         )

    #     fmri_array = np.concatenate((fmri_array, global_trend_array[None, :]), axis=0)
    #     assert np.equal(fmri_array[-1], global_trend_array).all()

    # fix so that the return for eeg are epochs 
    return epochs_array, fmri_array, fmri_output, eeg_channels, rois, eeg_path, fmri_path, eeg_times, events, event_ids, segment_scans


In [50]:

# 3. check last event for xtra tail 


In [51]:
def load_multiple_files(root: str,
                        rois: Optional[list[str]], eeg_channels: Optional[list[str]], sessions:list, separate_global_trend: bool,
                        rois_for_global_trend: Optional[list[str]] = None, roi_folder: str = 'roi'):
    datasets = {}
    event_ids_dict = {}
    if rois is None:
        overall_rois = None
    else:
        overall_rois = list(rois)

    if eeg_channels is None:
        overall_eeg_channels = None
    else:
        overall_eeg_channels = list(eeg_channels)

    for ses in sessions: 
        if ses == '1895':
            for task, run in RUNS_1895:
                #try:
                    epochs_array, fmri_array,fmri_output, eeg_channels, rois, eeg_path, fmri_path, eeg_times, events, event_ids, segment_scans = load_data(
                        root=root, ses=ses, run=run, task=task, rois=rois,
                        eeg_channels=eeg_channels, sampling_rates_ratio=500,
                        roi_folder=roi_folder, separate_global_trend=True
                    )

                    if overall_rois is None:
                        overall_rois = list(rois)
                    else:
                        assert overall_rois == rois

                    if overall_eeg_channels is None:
                        overall_eeg_channels = list(eeg_channels)
                    else:
                        assert overall_eeg_channels == eeg_channels

                    datasets[(eeg_path, fmri_path)] = {
                        'eeg': epochs_array, 'fmri': fmri_array, 'fmri_reshape':fmri_output, 'task': task.split('_')[0], 'eeg_times': eeg_times,'events':events, 'event_ids':event_ids,
                        'segment_scans':segment_scans
                    }
                    
                    event_ids_dict[(task, run, ses)] = event_ids
            #except Exception as e:
                #print(e) 
        else:
            for task, run in RUNS:
                #try:
                    epochs_array, fmri_array,fmri_output, eeg_channels, rois, eeg_path, fmri_path, eeg_times, events, event_ids, segment_scans = load_data(
                        root=root, ses=ses, run=run, task=task, rois=rois,
                        eeg_channels=eeg_channels, sampling_rates_ratio=500,
                        roi_folder=roi_folder, separate_global_trend = True
                    )

                    if overall_rois is None:
                        overall_rois = list(rois)
                    else:
                        assert overall_rois == rois

                    if overall_eeg_channels is None:
                        overall_eeg_channels = list(eeg_channels)
                    else:
                        assert overall_eeg_channels == eeg_channels

                    datasets[(eeg_path, fmri_path)] = {
                        'eeg': epochs_array, 'fmri': fmri_array,'fmri_reshape':fmri_output, 'task': task.split('_')[0], 'eeg_times': eeg_times,'events':events, 'event_ids':event_ids,
                        'segment_scans':segment_scans
                    }
                    event_ids_dict[(task, run, ses)] = event_ids


    rois = list(rois)
    if separate_global_trend:
        rois.append(' Global Regressive Trend')

    return datasets, rois, eeg_channels, event_ids_dict

In [52]:
root = '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472'
sessions = ['1818', '1895']

datasets, rois, eeg_channels, event_ids_dict= load_multiple_files(root = root, sessions=['1818', '1895'],
                                                  rois = None, eeg_channels = None, 
                                                  rois_for_global_trend = None, roi_folder = 'roi', separate_global_trend=separate_global_trend)
     


Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-01_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138060  =      0.000 ...   552.240 secs...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2795110670.py:7: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)


Used Annotations descriptions: ['New Segment/', 'Pulse Artifact/R', 'Scanner/Scan Start']


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2795110670.py:7: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)


Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 1250 original time points ...
0 bad epochs dropped
() [[3.22118195e+09 3.22101229e+09 3.22231640e+09 ... 3.22163830e+09
  3.22599916e+09 3.22140750e+09]
 [3.22101229e+09 3.22084263e+09 3.22214667e+09 ... 3.22146861e+09
  3.22582924e+09 3.22123783e+09]
 [3.22231640e+09 3.22214667e+09 3.22345124e+09 ... 3.22277290e+09
  3.22713530e+09 3.22254203e+09]
 ...
 [3.22163830e+09 3.22146861e+09 3.22277290e+09 ... 3.22209471e+09
  3.22645619e+09 3.22186388e+09]
 [3.22599916e+09 3.22582924e+09 3.22713530e+09 ... 3.22645619e+09
  3.23082358e+09 3.22622505e+09]
 [3.22140750e+09 3.22123783e+09 3.22254203e+09 ... 3.22186388e+09
  3.22622505e+09 3.22163307e+09]]
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-02_eegMRbvCBbviiR250.vhdr...
Setting channel i

...

Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 1250 original time points ...
0 bad epochs dropped
() [[2.23904777e+09 2.24168136e+09 2.23690697e+09 ... 2.24023973e+09
  2.23740222e+09 2.24161079e+09]
 [2.24168136e+09 2.24431805e+09 2.23953804e+09 ... 2.24287472e+09
  2.24003388e+09 2.24424740e+09]
 [2.23690697e+09 2.23953804e+09 2.23476821e+09 ... 2.23809778e+09
  2.23526299e+09 2.23946754e+09]
 ...
 [2.24023973e+09 2.24287472e+09 2.23809778e+09 ... 2.24143232e+09
  2.23859330e+09 2.24280411e+09]
 [2.23740222e+09 2.24003388e+09 2.23526299e+09 ... 2.23859330e+09
  2.23575788e+09 2.23996336e+09]
 [2.24161079e+09 2.24424740e+09 2.23946754e+09 ... 2.24280411e+09
  2.23996336e+09 2.24417675e+09]]
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1895/eeg/sub-101472_ses-1895_task-socword_run-02_eegMRbvCBbviiR250.vhdr...
Setting chan

/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2795110670.py:7: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2795110670.py:7: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)


Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 1250 original time points ...
0 bad epochs dropped
() [[2.26397457e+09 2.26555430e+09 2.26701266e+09 ... 2.26483164e+09
  2.26879220e+09 2.26928592e+09]
 [2.26555430e+09 2.26713514e+09 2.26859452e+09 ... 2.26641197e+09
  2.27037530e+09 2.27086937e+09]
 [2.26701266e+09 2.26859452e+09 2.27005483e+09 ... 2.26787088e+09
  2.27183676e+09 2.27233114e+09]
 ...
 [2.26483164e+09 2.26641197e+09 2.26787088e+09 ... 2.26568903e+09
  2.26965110e+09 2.27014500e+09]
 [2.26879220e+09 2.27037530e+09 2.27183676e+09 ... 2.26965110e+09
  2.27362009e+09 2.27411486e+09]
 [2.26928592e+09 2.27086937e+09 2.27233114e+09 ... 2.27014500e+09
  2.27411486e+09 2.27460974e+09]]
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1895/eeg/sub-101472_ses-1895_task-socword_run-03_eegMRbvCBbviiR250.vhdr...
Setting chan

/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2795110670.py:7: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2795110670.py:7: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)


Not setting metadata
46 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 46 events and 1250 original time points ...
0 bad epochs dropped
() [[2.25935911e+09 2.25686888e+09 2.25476622e+09 ... 2.25679738e+09
  2.26039735e+09 2.25540138e+09]
 [2.25686888e+09 2.25438140e+09 2.25228105e+09 ... 2.25430998e+09
  2.25790598e+09 2.25291551e+09]
 [2.25476622e+09 2.25228105e+09 2.25018266e+09 ... 2.25220969e+09
  2.25580235e+09 2.25081653e+09]
 ...
 [2.25679738e+09 2.25430998e+09 2.25220969e+09 ... 2.25423856e+09
  2.25783444e+09 2.25284413e+09]
 [2.26039735e+09 2.25790598e+09 2.25580235e+09 ... 2.25783444e+09
  2.26143607e+09 2.25643780e+09]
 [2.25540138e+09 2.25291551e+09 2.25081653e+09 ... 2.25284413e+09
  2.25643780e+09 2.25145058e+09]]


In [53]:
root = '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472'
sessions = ['1804', '1818', '1895']
channels_dict_1804= {}
channels_dict_1818= {}
channels_dict_1895= {}

for task, run  in RUNS: 
    filename_1804= f'sub-101472_ses-{sessions[0]}_task-{task}_run-{run}_eegMRbvCBbviiR250.vhdr'
    filename_1818= f'sub-101472_ses-{sessions[1]}_task-{task}_run-{run}_eegMRbvCBbviiR250.vhdr'

    eeg_vhdr_path_1804 = os.path.join(root, f'ses-{sessions[0]}', 'eeg', filename_1804)
    eeg_vhdr_path_1818 = os.path.join(root, f'ses-{sessions[1]}', 'eeg', filename_1818)

    eeg_raw_1804 = mne.io.read_raw_brainvision(eeg_vhdr_path_1804, preload = True)
    eeg_raw_1818 = mne.io.read_raw_brainvision(eeg_vhdr_path_1818, preload = True)

    df_eeg_1804 = eeg_raw_1804.to_data_frame()
    df_eeg_1818 = eeg_raw_1818.to_data_frame()

    channels_dict_1804[(task, run)] = df_eeg_1804.columns
    channels_dict_1818[(task, run)] = df_eeg_1818.columns


RUNS_1895 = [
    ('rs', '01'),
    ('rs', '02'),
    ('socword', '01'), # FAULTY FILE in eeg ses-1804 and ses-1895
    ('socword', '02'),
    ('socword', '03'),
    ]
for task, run  in RUNS_1895: 
    filename_1895= f'sub-101472_ses-{sessions[2]}_task-{task}_run-{run}_eegMRbvCBbviiR250.vhdr'

    eeg_vhdr_path_1895 = os.path.join(root, f'ses-{sessions[2]}', 'eeg', filename_1895)

    eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)

    df_eeg_1895 = eeg_raw_1895.to_data_frame()

    channels_dict_1895[(task, run)] = df_eeg_1895.columns


    

Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1804/eeg/sub-101472_ses-1804_task-rs_run-01_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138063  =      0.000 ...   552.252 secs...
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-01_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138060  =      0.000 ...   552.240 secs...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2664388218.py:14: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1804 = mne.io.read_raw_brainvision(eeg_vhdr_path_1804, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2664388218.py:14: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw_1804 = mne.io.read_raw_brainvision(eeg_vhdr_path_1804, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2664388218.py:15: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1818 = mn

Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1804/eeg/sub-101472_ses-1804_task-rs_run-02_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138062  =      0.000 ...   552.248 secs...
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-02_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138059  =      0.000 ...   552.236 secs...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2664388218.py:14: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1804 = mne.io.read_raw_brainvision(eeg_vhdr_path_1804, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2664388218.py:14: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw_1804 = mne.io.read_raw_brainvision(eeg_vhdr_path_1804, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2664388218.py:15: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1818 = mn

Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1804/eeg/sub-101472_ses-1804_task-rs_run-03_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138068  =      0.000 ...   552.272 secs...
Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-03_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138055  =      0.000 ...   552.220 secs...


...

/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2664388218.py:36: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2664388218.py:36: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)


Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1895/eeg/sub-101472_ses-1895_task-socword_run-02_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138057  =      0.000 ...   552.228 secs...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2664388218.py:36: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2664388218.py:36: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)


Extracting parameters from /Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1895/eeg/sub-101472_ses-1895_task-socword_run-03_eegMRbvCBbviiR250.vhdr...
Setting channel info structure...
Reading 0 ... 138024  =      0.000 ...   552.096 secs...


/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2664388218.py:36: RuntimeWarning: No coordinate information found for channels ['ECG']. Setting channel types to misc. To avoid this warning, set channel types explicitly.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)
/var/folders/_t/wzqtw10969z1swmc_n6s3p700000gp/T/ipykernel_12783/2664388218.py:36: RuntimeWarning: Not setting position of 1 misc channel found in montage:
['ECG']
Consider setting the channel types to be of EEG/sEEG/ECoG/DBS/fNIRS using inst.set_channel_types before calling inst.set_montage, or omit these channels when creating your montage.
  eeg_raw_1895 = mne.io.read_raw_brainvision(eeg_vhdr_path_1895, preload = True)


In [54]:
## TEST INDIVIDUAL FUNCTIONS 
# root = '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472'
# eeg_channels = None
# sampling_rates_ratio = 500
# leeway = 0.001
# filename = f'sub-101472_ses-1818_task-socword_run-06_eegMRbvCBbviiR250.vhdr'
# eeg_vhdr_path = os.path.join(root, f'ses-1818', 'eeg', filename)
# eeg_raw = mne.io.read_raw_brainvision(eeg_vhdr_path, preload = True)

# events, event_ids = mne.events_from_annotations(eeg_raw)
# events_dict_cut = {'Scanner/Scan Start': event_ids['Scanner/Scan Start']}  # use only Scan Start event



# try:
#     scan_key = 'Scanner/Scan Start'
#     segment_scans = events[events[:, -1] == event_ids[scan_key]]
# except: 
#     scan_key = 'Stimulus/Scan Start'
#     segment_scans = events[events[:, -1] == event_ids[scan_key]] 
#     print(f'ERROR: FAULTY EVENT NAME. PATH:{eeg_vhdr_path}')


# unique_scan_diffs, counts_diffs = np.unique(np.diff(segment_scans[:, 0]), return_counts=True)

# first_segment = segment_scans[0, 0] / EEG_SAMPLING_RATE
# cropped_eeg = eeg_raw.crop(tmin=first_segment)

# df_eeg = eeg_raw.to_data_frame()
# cropped_df_eeg = cropped_eeg.to_data_frame()

# if eeg_channels is None:
#     eeg_channels = [column for column in df_eeg.columns if column != 'time']

# eeg_times = df_eeg['time']
# cropped_eeg_times = cropped_df_eeg['time']

# df_eeg = df_eeg[eeg_channels]


# fmri_array, fmri_times, rois, fmri_path, df_fmri = load_fmri_data(
#         root=root, run='01', ses='1818', task='rs', rois = None, roi_folder='roi'
#     )





# n_eeg_samples = df_fmri.shape[0] * sampling_rates_ratio
# df_eeg_new = df_eeg.iloc[:n_eeg_samples]
# eeg_array = df_eeg_new.to_numpy().T

# epochs = mne.Epochs(
#         cropped_eeg,
#         events = segment_scans, 
#         event_id =events_dict_cut, 
#         tmin = 6.5,
#         tmax = 11.5-1/EEG_SAMPLING_RATE, 
#         baseline = None, 
#         preload = True
#     )
# print(epochs.drop_log)

# epochs_df = epochs.to_data_frame()
# #epochs_df = epochs_df[:-1]
# segment_scans = segment_scans[:-1]

# #epochs = epochs.copy().filter(1, 70)
# #epochs = epochs.crop(tmin=6 + leeway, tmax=12 - leeway, include_tmax=False)


In [55]:
# # assert for tsart integer 
# all([i.is_integer() for i in df_fmri[' Tstart']])
# fmri_times = fmri_times.astype(int)
# fmri_times

In [56]:
def split_single_dataset(eeg: np.ndarray, fmri: np.ndarray, task: str): 
    
    # зафиксировать by % test and val, put the rest in train
    
    eeg_train, eeg_test, fmri_train, fmri_test = train_test_split(
        eeg, fmri, test_size=0.25, shuffle=False
        )

    eeg_train, eeg_val, fmri_train, fmri_val = train_test_split(
        eeg_train, fmri_train, test_size=0.2, shuffle=False
        ) 

    train_data = {'eeg': eeg_train,
                  'fmri': fmri_train}

    val_data = {'eeg': eeg_val,
                'fmri': fmri_val}

    test_data = {'eeg': eeg_test,
                 'fmri': fmri_test}

    return train_data, val_data, test_data

def split_datasets(datasets: dict, separate_global_trend:bool):
    train_datasets = {}
    validation_datasets = {}
    test_datasets = {}

    for set_path, single_dataset in datasets.items():

        if separate_global_trend: 
            train_data, val_data, test_data = split_single_dataset(
            eeg=single_dataset['eeg'], fmri=single_dataset['fmri_reshape'],
            task=single_dataset['task']
        )
        else: 
            train_data, val_data, test_data = split_single_dataset(
                eeg=single_dataset['eeg'], fmri=single_dataset['fmri'],
                task=single_dataset['task']
            )

        train_datasets[set_path] = train_data

        if val_data is not None:
            validation_datasets[set_path] = val_data

        if test_data is not None:
            test_datasets[set_path] = test_data

    return train_datasets, validation_datasets, test_datasets




In [57]:
train_datasets, validation_datasets, test_datasets = split_datasets(datasets, separate_global_trend=separate_global_trend)

In [58]:
validation_datasets[('/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-01_eegMRbvCBbviiR250.vhdr', '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/roi/sub-101472_ses-1818_task-rs_run-01.csv')]['eeg'].shape

(7, 127, 1250)

In [59]:
validation_datasets[('/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-01_eegMRbvCBbviiR250.vhdr', '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/roi/sub-101472_ses-1818_task-rs_run-01.csv')]['fmri'].shape

(7, 23, 3)

In [60]:
# initialise for each run and then concatDataset <- rewrite from build_full_dataset 
class EEGFMRIDataset(Dataset):
    def __init__(self, eeg, fmri, data_path, eeg_channels, fmri_rois, mean=False ):
        self.data_path = data_path
        self.eeg = torch.from_numpy(eeg).float()
        if mean: 
            # keepdims so that the last dimension will be 1 
            fmri = np.mean(fmri, axis = -1, keepdims=True)
        self.fmri = torch.from_numpy(fmri).float()

        self.eeg_channels = list(eeg_channels)
        self.fmri_rois = list(fmri_rois)

        assert self.eeg.size(0) == self.fmri.size(0), (self.eeg.size(), self.fmri.size())

    def __getitem__(self, idx):
        return self.eeg[idx, ...], self.fmri[idx, ...]

    def __len__(self):
        return self.eeg.size(0)
    
    def get_rois(self):
        return list(self.fmri_rois)
    
def build_full_dataset(datasets:dict, fmri_rois:list[str], mean:bool, separate_global_trend:bool):
    torch_datasets = []
    all_rois = None
    for ds_path, dataset in datasets.items():
        torch_dataset = EEGFMRIDataset(
            eeg=dataset['eeg'], fmri=dataset['fmri'],
            eeg_channels=eeg_channels, fmri_rois=fmri_rois,
            data_path=ds_path, mean=mean)
        torch_rois = torch_dataset.get_rois()
        if all_rois is None:
            all_rois = list(torch_rois)
        else:
            assert all_rois == torch_rois, (all_rois, torch_rois)
        torch_datasets.append(torch_dataset)
    full_dataset = ConcatDataset(torch_datasets)

    return full_dataset, all_rois


In [61]:
datasets[('/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/eeg/sub-101472_ses-1818_task-rs_run-01_eegMRbvCBbviiR250.vhdr',
  '/Users/terekhova/Desktop/EEG-BOLD-Decoding/intermittent/interm/sub-101472/ses-1818/roi/sub-101472_ses-1818_task-rs_run-01.csv')]['eeg'].shape

(45, 127, 1250)

In [62]:
torch_datasets = {}
all_datasets = {'train': train_datasets, 'validation': validation_datasets, 'test': test_datasets}

for split_name in ['train', 'validation', 'test']:

    split_dataset, split_rois = build_full_dataset(
        datasets=all_datasets[split_name], fmri_rois=rois, mean = True, separate_global_trend=separate_global_trend)

    torch_datasets[split_name] = {'dataset': split_dataset, 'rois': split_rois}

In [63]:
train_dataset, validation_dataset, test_dataset = torch_datasets['train']['dataset'], torch_datasets['validation']['dataset'], torch_datasets['test']['dataset']


In [64]:
validation_dataset.datasets[2].fmri.shape


torch.Size([7, 23, 1])

In [65]:
model_config = {"dropout_prob": 0.25, "intermediate_channels": None}
scheduler_config = {"mode": "min", "factor": 0.1, "patience": 5}

In [66]:
# brain_wave_linker_net = BrainWaveLinker(
#         nn_parameters=model_config,
#         eeg_channel_names=eeg_channels,
#         roi_names=rois,
#         criterion_name='MSELoss',
#         criterion_kwargs={"dim": -1, "eps": 1e-06},
#         optimizer_name='Adam',
#         optimizer_kwargs={"lr": 0.001,"weight_decay": 0.001},
#         scheduler_kwargs=scheduler_config
#     )

In [67]:
def build_dataloader(dataset: torch.utils.data.Dataset, sampler_parameters: Optional[dict],
                     dataloader_parameters: dict):

    samplers = {
        'RandomSampler': RandomSampler
    }

    if sampler_parameters is not None:
        sampler = samplers[sampler_parameters['name']](dataset, **sampler_parameters['parameters'])
    else:
        sampler = None

    dataloader = torch.utils.data.DataLoader(dataset, sampler=sampler, **dataloader_parameters)

    return dataloader


In [68]:
dataloader_configs = {"train": {"batch_size": 128,"drop_last": False,"num_workers": 0},  "validation": {
        "batch_size": 128,
        "drop_last": False,
        "num_workers": 0
    },
    "test": {
        "batch_size": 128,
        "drop_last": False,
        "num_workers": 0
    }
}

sampler_configs = {
        "train": None,
        "validation": None
    }

In [69]:
train_dataset.datasets[0].fmri.shape

torch.Size([26, 23, 1])

In [70]:
samplers = {'RandomSampler': RandomSampler}

train_loader = torch.utils.data.DataLoader(train_dataset, sampler=None, batch_size = 128, drop_last=False, num_workers=0)
val_loader = torch.utils.data.DataLoader(validation_dataset, sampler=None, batch_size = 128, drop_last=False, num_workers=0)
test_loader = torch.utils.data.DataLoader(train_dataset, sampler=None, batch_size = 128, drop_last=False, num_workers=0)





In [71]:
# train_loader = build_dataloader(dataset=train_dataset, sampler_parameters=sampler_configs['train'],
#                                     dataloader_parameters=dataloader_configs['train'])
# val_loader = build_dataloader(dataset=validation_dataset, sampler_parameters=sampler_configs['validation'],
#                                   dataloader_parameters=dataloader_configs['validation'])
# test_loader = build_dataloader(dataset=test_dataset, sampler_parameters=None,
#                                    dataloader_parameters=dataloader_configs['test'])

In [72]:
print('train_loader', len(train_loader))
print('val_loader', len(val_loader))
print('test_loader', len(test_loader))

train_loader 4
val_loader 1
test_loader 4


In [73]:
trainer_config= {
        "early_stopping": {
            "monitor": "val_loss",
            "mode": "min",
            "patience": 20
        },
        "checkpointing": {
            "monitor": "val_loss",
            "mode": "min",
            "save_top_k": 1
        },
        "kwargs": {
            "max_epochs": 500,
            "log_every_n_steps": 1
        }
    }

In [74]:
train_dataset.datasets[1].fmri.shape

torch.Size([26, 23, 1])

In [75]:
# for i, (inputs, targets) in enumerate (train_loader): 
#     print(f"Batch {i}:")
#     print(f"Inputs shape: {inputs.shape}")
#     print(f"Targets shape: {targets.shape}")

In [76]:
# for batch in train_loader:
#         inputs, targets = batch
#         outputs = brain_wave_linker_net(inputs)  
#         print(inputs.shape, outputs.shape, targets.shape)

In [77]:
# Keep in mind entry and final dimensions 
# if returning to 1-point prediction, put temporal filter = 10 


def build_criterion(criterion_name: str, criterion_kwargs: dict):

    if criterion_name == 'MSELoss':
        criterion = nn.MSELoss(**criterion_kwargs)
    elif criterion_name == 'L1Loss':
        criterion = nn.MSELoss(**criterion_kwargs)
    else:
        raise NotImplementedError
    return criterion

class BrainWaveLinker(nn.Module):
    
    def __init__(
            self, in_channels: int, out_channels: int, dropout_prob: float = 0.25,
            intermediate_channels: Optional[int] = None
    ):
        super().__init__()

        if intermediate_channels is None:
            intermediate_channels = out_channels

        self.spatial_filer = nn.Conv1d(in_channels,  intermediate_channels, kernel_size=1)

        self.pyramidal_subsampling = nn.Sequential(
            nn.GELU(),
            nn.Conv1d(in_channels=intermediate_channels, out_channels=intermediate_channels,
                      kernel_size=5, stride=5),
            nn.GELU(),
            nn.Dropout(dropout_prob),
            nn.Conv1d(in_channels=intermediate_channels, out_channels=intermediate_channels,
                      kernel_size=5, stride=5),
            nn.GELU(),
            nn.Dropout(dropout_prob),
            nn.Conv1d(in_channels=intermediate_channels, out_channels=intermediate_channels,
                      kernel_size=5, stride=5),
            nn.GELU(),
            nn.Dropout(dropout_prob)
        )

        self.temporal_filter = nn.Sequential(
            nn.Conv1d(in_channels=intermediate_channels, out_channels=out_channels, 
                      kernel_size = 10)
        )
        # self.temporal_filter = nn.Conv1d(
        #     intermediate_channels, out_channels, kernel_size=8, padding=0
        # )

    def forward(self, x: torch.Tensor):
        out_sf = self.spatial_filer(x)
        out_ps = self.pyramidal_subsampling(out_sf)
        out = self.temporal_filter(out_ps)
        return out


class BrainWaveLinkerSystem(L.LightningModule):
    
    def __init__(self,
                 nn_parameters: dict,
                 roi_names: list[str], eeg_channel_names: list[str],
                 criterion_name: str, criterion_kwargs: dict,
                 optimizer_name: str, optimizer_kwargs: dict,
                 scheduler_kwargs: dict):
        super().__init__()

        self.model = BrainWaveLinker(
            in_channels=len(eeg_channel_names),
            out_channels=len(roi_names),
            **nn_parameters
        )
        self.criterion = build_criterion(criterion_name=criterion_name, criterion_kwargs=criterion_kwargs)
        self.eeg_channel_names = list(eeg_channel_names)
        self.roi_names = list(roi_names)
        self.criterion_name = criterion_name
        self.criterion_kwargs = criterion_kwargs
        self.optimizer_name = optimizer_name
        self.optimizer_kwargs = optimizer_kwargs
        self.scheduler_name = 'ReduceLROnPlateau'
        self.scheduler_kwargs = scheduler_kwargs


    def forward(self, inputs: torch.Tensor):
        return self.model(inputs)


    def common_step(self, batch: tuple[torch.Tensor], batch_idx: int, mode: str):
        inputs, target = batch
        output = self(inputs)
        loss = self.criterion(output, target)
 
        self.log(f'{mode}_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)

        return loss

    def training_step(self, batch: tuple[torch.Tensor], batch_idx: int):
        loss = self.common_step(batch=batch,batch_idx=batch_idx,mode='train')
        return loss

    def validation_step(self, batch: tuple[torch.Tensor], batch_idx: int):
        loss = self.common_step(batch=batch,batch_idx=batch_idx,mode='val')
        return loss

    def test_step(self, batch: tuple[torch.Tensor], batch_idx: int):
        loss  = self.common_step(batch=batch,batch_idx=batch_idx,mode='test')
        return loss


    def configure_optimizers(self):
        optimizers = {
            'Adam': torch.optim.Adam,
            'AdamW': torch.optim.AdamW,
            'RMSprop': torch.optim.RMSprop,
            'SGD': torch.optim.SGD
        }
        optimizer = optimizers[self.optimizer_name](self.model.parameters(), **self.optimizer_kwargs)
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, **self.scheduler_kwargs),
                # ReduceLROnPlateau scheduler requires a monitor
                'monitor': 'val_loss',
                'frequency': 1
            },
        }
def build_dataloader(dataset: torch.utils.data.Dataset, sampler_parameters: Optional[dict],
                     dataloader_parameters: dict):

    samplers = {
        'RandomSampler': RandomSampler
    }

    if sampler_parameters is not None:
        sampler = samplers[sampler_parameters['name']](dataset, **sampler_parameters['parameters'])
    else:
        sampler = None

    dataloader = torch.utils.data.DataLoader(dataset, sampler=sampler, **dataloader_parameters)

    return dataloader

def run_full_pipeline(trainer_config, brain_wave_linker_net):


    callbacks = [ModelCheckpoint(**trainer_config['checkpointing'])]

    if trainer_config['early_stopping'] is not None:
        callbacks.append(EarlyStopping(monitor=trainer_config['early_stopping']['monitor'],
                                       mode=trainer_config['early_stopping']['mode'],
                                       patience=trainer_config['early_stopping']['patience']))

    
    trainer = L.Trainer(callbacks=callbacks,
                        **trainer_config['kwargs'])

    best_model = None



    trainer.fit(model=brain_wave_linker_net, train_dataloaders=train_loader, val_dataloaders=val_loader)
    print(trainer.checkpoint_callback)
    best_model = trainer.checkpoint_callback.best_model_path  # Assuming this retrieves the best model's state_dict


    print(f'Training complete')


    

    trainer.test(brain_wave_linker_net, dataloaders=test_loader, ckpt_path = best_model)    

    # for logging_mode in config['dataframe_logging_modes']:
    #     brain_wave_linker_net.log_metrics_tables(mode=logging_mode)

    if torch.cuda.is_available():
        device = 'cuda'
    else:
        device = 'cpu'
        
    # extract_predicted_time_series(
    #     model=brain_wave_linker_net.model, ckpt_path=best_model_path, dataset=test_loader.dataset, ds_name='test',
    #     device=device
    # )


    # print_wrapped_text(header_text='MODEL', main_body=brain_wave_linker_net.model)
    # print_wrapped_text(header_text='Experiment config', main_body=config)
    # print_wrapped_text(header_text='Criterion config', main_body=criterion_config)
    # print_wrapped_text(header_text='Optimizer config', main_body=optimizer_config)
    # print_wrapped_text(header_text='Scheduler config', main_body=scheduler_config)
    # print_wrapped_text(header_text='Trainer config', main_body=trainer_config)
    # print_wrapped_text(header_text='wandb config', main_body=wandb_kwargs)
    # print_wrapped_text(header_text='Model config', main_body=model_config)


In [78]:
# def build_dataloader(dataset: torch.utils.data.Dataset, sampler_parameters: Optional[dict],
#                      dataloader_parameters: dict):

#     samplers = {
#         'RandomSampler': RandomSampler
#     }

#     if sampler_parameters is not None:
#         sampler = samplers[sampler_parameters['name']](dataset, **sampler_parameters['parameters'])
#     else:
#         sampler = None

#     dataloader = torch.utils.data.DataLoader(dataset, sampler=sampler, **dataloader_parameters)

#     return dataloader


In [79]:

# def extract_predicted_time_series(model: torch.nn.Module, ckpt_path: str, dataset: torch.utils.data.ConcatDataset,
#                                   ds_name: str, device: str):
#     # Load checkpoint
#     checkpoint = torch.load(ckpt_path, weights_only=True, map_location=device)
#     state_dict = {}

#     # Original checkpoint for a torch.nn.Module was inside a lightning module, therefore removing additional prefix
#     for key, value in checkpoint['state_dict'].items():
#         key = key.split('.')
#         assert key[0] == 'model'
#         new_key = '.'.join(key[1:])
#         state_dict[new_key] = value
#     model.load_state_dict(state_dict)
#     model.eval()

#     # Check ROI consistency
#     roi_names = None
#     for ds in dataset.datasets:
#         if roi_names is None:
#             roi_names = list(ds.get_rois())
#         else:
#             assert roi_names == list(ds.get_rois()), (roi_names, ds.get_rois())


#     for ds in dataset.datasets:

#         eeg, gt_fmri, (eeg_data_path, fmri_data_path) = ds.get_all_data()
#         eeg = eeg.unsqueeze(0)  # imitating batch dimension
#         eeg.to(device)

#         with torch.no_grad():
#             pred_fmri = model(eeg)
#             assert pred_fmri.size(0) == 1  # remove batch dimension
#             pred_fmri = pred_fmri[0, ...]

#         pred_fmri = pred_fmri.cpu().numpy()
#         gt_fmri = gt_fmri.cpu().numpy()

        

#         gt_fmri = gt_fmri[:, :-20]

#         ds_results = {}

#         for roi_idx, roi_name in enumerate(roi_names):
#             roi_pred = pred_fmri[roi_idx, :]
#             roi_gt = gt_fmri[roi_idx, :]
#             ds_results[f'{roi_name}_pred'] = roi_pred
#             ds_results[f'{roi_name}_gt'] = roi_gt


#         ds_results = pd.DataFrame(ds_results)



In [80]:
brain_wave_linker_net = BrainWaveLinkerSystem(
        nn_parameters=model_config,
        eeg_channel_names=eeg_channels,
        roi_names=rois,
        criterion_name='MSELoss',
        criterion_kwargs={},
        optimizer_name='Adam',
        optimizer_kwargs={"lr": 0.001,"weight_decay": 0.001},
        scheduler_kwargs=scheduler_config)

In [81]:
type(brain_wave_linker_net)

__main__.BrainWaveLinkerSystem

In [82]:
train_dataset.datasets[0].fmri.shape

torch.Size([26, 23, 1])

In [83]:
run_full_pipeline(trainer_config=trainer_config, brain_wave_linker_net=brain_wave_linker_net)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name      | Type            | Params | Mode 
------------------------------------------------------
0 | model     | BrainWaveLinker | 16.3 K | train
1 | criterion | MSELoss         | 0      | train
------------------------------------------------------
16.3 K    Trainable params
0         Non-trainable params
16.3 K    Total params
0.065     Total estimated model params size (MB)
16        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/terekhova/fsl/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


/Users/terekhova/fsl/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 49: 100%|██████████| 4/4 [00:00<00:00, 21.85it/s, v_num=36, train_loss_step=9.82e+7, val_loss_step=1.26e+8, val_loss_epoch=1.26e+8, train_loss_epoch=1.26e+8]

Restoring states from the checkpoint path at /Users/terekhova/Desktop/github /lightning_logs/version_36/checkpoints/epoch=29-step=120.ckpt



Training complete


Loaded model weights from the checkpoint at /Users/terekhova/Desktop/github /lightning_logs/version_36/checkpoints/epoch=29-step=120.ckpt
/Users/terekhova/fsl/lib/python3.12/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 35.29it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      test_loss_epoch      │        125634912.0        │
└───────────────────────────┴───────────────────────────┘

In [ ]:
# use MAPE 
# try plotting the graphs - try imagescale, try plooting seevral time series on one time axis 
    # try spider 
# remove the global trend (?)
# how to preprocess the data with the ортогональая проекци: via numpy reshape сжать и разжать в иначальное ищмерение 

# посмотреть на magnitude  

# fmri prep показать что у одного и того же человека меняется активация roi при стимуле; потом проверить виде ли тот же резлуьтат когда в 2 рааза меньше данных 
# possibly imterpolate дфрки в данных или просто убрать пробелы в данных и в коонтрасте 

In [ ]:
rois 

[' Left Cerebral White Matter',
 ' Left Cerebral Cortex',
 ' Left Lateral Ventricle',
 ' Left Thalamus',
 ' Left Caudate',
 ' Left Putamen',
 ' Left Pallidum',
 ' Left Brain-Stem',
 ' Right Brain-Stem',
 ' Left Hippocampus',
 ' Left Amygdala',
 ' Left Accumbens',
 ' Right Cerebral White Matter',
 ' Right Cerebral Cortex',
 ' Right Lateral Ventricle',
 ' Right Thalamus',
 ' Right Caudate',
 ' Right Putamen',
 ' Right Pallidum',
 ' Right Hippocampus',
 ' Right Amygdala',
 ' Right Accumbens']